In [15]:
%load_ext autoreload
%autoreload 2

import os
curpath = os.getcwd()
os.chdir(curpath.split("core")[0])

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
from openai import OpenAI
import json
import time
import asyncio
from typing import List, Dict, Any, Tuple, Union

from core.features.content_development.python.prompts import SIMPLE_COURSE_GENERATION_PROMPT, JUPYTER_NOTEBOOK_TUTORIAL_PROMPT
from core.features.topic_segregation.utils import add_dicts, calculate_cost_gpt4_8k, calculate_cost_gpt4_turbo
from core.features.provider import async_creator, text_model_defaults

from dotenv import load_dotenv
load_dotenv()

client = OpenAI()

In [ ]:
"""
FLOW - 
1. generate a course structure after asking for a topic, scope of the topic, duration of course (in hours), parts (modules)
format ```json 

{
    "topic": "topic",
    "curriculum": {
        "module1" : [list of subtopics in module1],
        "module2" : [list of subtopics in module2],
            ...
    },
}

2. take first topic of module1, and ask for the content on it with proper comments and explanations. add all the response in different files and iterate for all the topics of all the modules. 


pseudo code - 


structure = generate_structure(topic, topic_scope, duration_in_hours, number_of_parts)

for module in structure["curriculum"]:
    for topic in structure["curriculum"][module]:
        content = generate_content(topic)
        save_content(content)


"""

In [ ]:
async def generate_course_structure(title:str, scope:str, duration:int, num_modules:int, format=None):

    conversation = [{"role": "system", "content": SIMPLE_COURSE_GENERATION_PROMPT}]

    user_input = f"""
    topic: {title}
    scope: {scope}
    duration: {duration}
    num_modules: {num_modules}
    """

    user_message = {'role': 'user', 'content': user_input}
    conversation.append(user_message)

    response = await async_creator(
        **text_model_defaults,
        messages=conversation
        )
    
    output = response.choices[0].message.content
    total_usage = response.usage

    return {"output": output, "total_usage": total_usage}


async def generate_content_notebook(curriculum, previous_part, current_topic):

    conversation = [{"role": "system", "content": JUPYTER_NOTEBOOK_TUTORIAL_PROMPT}]

    user_input = f"""
    curriculum = {curriculum},

    previous_part = {previous_part},

    current_part = {current_topic}
    """

    user_message = {'role': 'user', 'content': user_input}
    conversation.append(user_message)

    response = await async_creator(
        **text_model_defaults,
        messages=conversation
        )
    
    output = response.choices[0].message.content
    total_usage = response.usage

    return {"output": output, "total_usage": total_usage}


def convert_to_jupyter_notebook(input_json, file_name):

    # Template for a Jupyter Notebook

    """
    Rules to create the jupyter notebook:
    1. metadata tag must be present in code cell
    """
    notebook_template = {
        "cells": [],
        "metadata": {'kernelspec': {'display_name': 'Python 3',
            'language': 'python',
            'name': 'python3'},
            'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
            'file_extension': '.py',
            'mimetype': 'text/x-python',
            'name': 'python',
            'nbconvert_exporter': 'python',
            'pygments_lexer': 'ipython3',
            'version': '3.8.5'}},
        "nbformat": 4,
        "nbformat_minor": 4
    }

    # Adding the cells from the input JSON to the notebook template
    for cell in input_json['cells']:
        if cell['cell_type'] == "code":
            cell['metadata'] = {}

    notebook_template['cells'] = input_json['cells']

    with open(f'{file_name}.json', 'w') as file:
        json.dump(notebook_template, file, indent=2)

        
result = asyncio.run(generate_course_structure(title="Generative Adversarial Networks with Tensorflow", scope="Neural Networks to GANs", duration=10, num_modules=4))


In [ ]:
async def content_pipeline(title, curriculum, path):

    previous_part = "NA"
    for module, topics in curriculum['curriculum'].items():
        for topic in topics:
            # Assuming generate_content_notebook and convert_to_jupyter_notebook are asynchronous
            result = await generate_content_notebook(curriculum=curriculum, previous_part=previous_part, current_topic=topic)
            content_json = json.loads(result['output'])

            convert_to_jupyter_notebook(content_json, os.path.join(path, title.replace(" ", "_"), module, topic.replace(" ", "_") + ".ipynb"))

            previous_part = content_json

        print("Module" + str(i+1) + " of " + str(len(curriculum['curriculum'])) + " completed")
    print("Content pipeline completed")


title = result['output']['title']
curriculum = result['output']['curriculum']

asyncio.run(content_pipeline(title=title, curriculum=curriculum, path="content_development"))

In [19]:
raw_notebook = json.loads(result['output'])

curriculum = raw_notebook

curriculum

{'title': 'Generative Adversarial Networks with Tensorflow',
 'curriculum': {'module1': ['Introduction to Neural Networks',
   'Fundamentals of Deep Learning',
   'Activation Functions',
   'Loss Functions',
   'Backpropagation and Gradient Descent',
   'Tensorflow Basics'],
  'module2': ['Convolutional Neural Networks (CNNs)',
   'CNN Architectures',
   'Tensorflow for CNNs',
   'Image Processing and Generation',
   'Feature Extraction and Transfer Learning'],
  'module3': ['Introduction to Generative Models',
   'Autoencoders',
   'Variational Autoencoders (VAEs)',
   'Introduction to Generative Adversarial Networks (GANs)'],
  'module4': ['GAN Architectures and Applications',
   'Designing and Training GANs with Tensorflow',
   'Advanced GANs (DCGAN, CGAN, etc.)',
   'Challenges in Training GANs',
   'Project: Building and Training a GAN with Tensorflow']}}

In [20]:

def generate_content_notebook(curriculum, previous_part, current_topic):

    conversation = [{"role": "system", "content": JUPYTER_NOTEBOOK_TUTORIAL_PROMPT}]

    user_input = f"""
    curriculum = {curriculum},

    previous_part = {previous_part},

    current_part = {current_topic}
    """

    user_message = {'role': 'user', 'content': user_input}
    conversation.append(user_message)

    response = creator(
        **text_model_defaults,
        messages=conversation
        )
    
    output = response.choices[0].message.content
    total_usage = response.usage

    return {"output": output, "total_usage": total_usage}


result = generate_content_notebook(curriculum=curriculum, previous_part="NA", current_topic="Introduction to Neural Networks")


In [21]:
output = json.loads(result['output'])
output

{'cells': [{'cell_type': 'markdown',
   'source': '# Introduction to Neural Networks\n\nIn this section, we will dive into the foundational concepts of neural networks, which are the building blocks of deep learning and the technology behind many modern artificial intelligence applications.'},
  {'cell_type': 'markdown',
   'source': '## What is a Neural Network?\n\nA neural network is a computational model inspired by the way biological neural networks in the human brain process information. It consists of a large number of interconnected processing elements called neurons that work together to solve specific problems. Neural networks are used for a variety of tasks, such as image and speech recognition, language translation, and playing games, among others.'},
  {'cell_type': 'markdown',
   'source': "## Components of a Neural Network\n\n- **Neurons**: The basic unit of a neural network, often called a node or perceptron, which receives input, processes it, and generates an output.\n

In [23]:

result = generate_content_notebook(curriculum=curriculum, previous_part=output, current_topic='Fundamentals of Deep Learning')
output = json.loads(result['output'])
output

{'cells': [{'cell_type': 'markdown',
   'source': "# Fundamentals of Deep Learning\n\nDeep learning is a subset of machine learning that uses neural networks with many layers (hence 'deep') to model complex patterns in data. In this section, we will explore the core concepts that underpin deep learning, including the architecture of deep networks, the role of data, and the algorithms that drive learning."},
  {'cell_type': 'markdown',
   'source': '## What is Deep Learning?\n\nDeep learning is a machine learning technique that teaches computers to learn by example, just as humans do. It is a key technology behind driverless cars, enabling them to recognize a stop sign, or distinguishing a pedestrian from a lamppost. It is the key to voice control in consumer devices like phones, tablets, TVs, and hands-free speakers. Deep learning is getting lots of attention lately and for good reason. It’s achieving results that were not possible before.'},
  {'cell_type': 'markdown',
   'source': '#

In [28]:
usage = result['total_usage'].model_dump()

print(usage)

calculate_cost_gpt4_turbo(usage)

{'completion_tokens': 1112, 'prompt_tokens': 1291, 'total_tokens': 2403}


0.04627

In [24]:
import json


def convert_to_jupyter_notebook(input_json, file_name):

    # Template for a Jupyter Notebook

    """
    Rules to create the jupyter notebook:
    1. metadata tag must be present in code cell
    """
    notebook_template = {
        "cells": [],
        "metadata": {'kernelspec': {'display_name': 'Python 3',
            'language': 'python',
            'name': 'python3'},
            'language_info': {'codemirror_mode': {'name': 'ipython', 'version': 3},
            'file_extension': '.py',
            'mimetype': 'text/x-python',
            'name': 'python',
            'nbconvert_exporter': 'python',
            'pygments_lexer': 'ipython3',
            'version': '3.8.5'}},
        "nbformat": 4,
        "nbformat_minor": 4
    }

    # Adding the cells from the input JSON to the notebook template
    for cell in input_json['cells']:
        if cell['cell_type'] == "code":
            cell['metadata'] = {}

    notebook_template['cells'] = input_json['cells']

    with open(f'{file_name}.json', 'w') as file:
        json.dump(notebook_template, file, indent=2)


# Convert to Jupyter Notebook JSON
jupyter_notebook_json = convert_to_jupyter_notebook(output, file_name='deep_learning_fundamentals')